# Facial Emotion Recognition

In [ ]:
import os
import cv2

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


data_dir = "/content/train"
emotion_labels = ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised']

images = []
labels = []

for label_id, emotion in enumerate(emotion_labels):
    emotion_path = os.path.join(data_dir, emotion)
    for img_filename in os.listdir(emotion_path):
        img_path = os.path.join(emotion_path, img_filename)
        
        img = cv2.imread(img_path)
        if img is not None:  
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(label_id)



# Preparing Dataset


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

train_data = train_datagen.flow_from_directory(
    directory="/content/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)

val_data = val_datagen.flow_from_directory(
    directory="/content/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


# Model Selection

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping



base_model = MobileNet(input_shape=(224, 224, 3), include_top=False)

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(units=7, activation='softmax')(x)

model = Model(base_model.input, x)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=categorical_crossentropy,metrics=['accuracy'])

In [ ]:
class_weights = {0: 7215/7.0, 1: 4965/7.0, 2: 4830/7.0, 3: 3171/7.0, 4: 3995/7.0, 5: 4097/7.0, 6: 436/7.0}

es = EarlyStopping(monitor='val_accuracy', min_delta=0.01, patience=5, verbose=1, mode='auto')
mc = ModelCheckpoint(filepath="emotion_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')


In [ ]:

hist = model.fit(
    train_data,
    steps_per_epoch=len(train_data),
    epochs=30,
    validation_data=val_data,
    validation_steps=len(val_data),
    callbacks=[es, mc],
    class_weight=class_weights
)


# Model Validation


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import confusion_matrix


In [ ]:
validation_labels = val_data.classes
validation_pred_probs = model.predict(val_data)
validation_pred_labels = np.argmax(validation_pred_probs, axis=1)

confusion_mtx = confusion_matrix(validation_labels, validation_pred_labels)
class_names = list(train_data.class_indices.keys())
sns.set()
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='YlGnBu', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'] , c = "red")
plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'] , c = "red")
plt.title("loss vs v-loss")
plt.show()